In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [23]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import warnings
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb


import eli5
from eli5.sklearn import PermutationImportance
from eli5.permutation_importance import get_score_importances

In [3]:
import pandas as pd # 데이터 분석
import numpy as np # 행렬 연산, version: 1.6.1

import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import sklearn
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD, PCA, FastICA, FactorAnalysis, KernelPCA, DictionaryLearning
from sklearn.decomposition import IncrementalPCA, LatentDirichletAllocation,MiniBatchSparsePCA, SparsePCA

import itertools

In [4]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
sns.set(font_scale=2.5)

warnings.filterwarnings('ignore')

# 데이터 로드

In [5]:
data_dir = Path('../input/dankook')
sub_dir = Path('../output/')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'
sub_file = sub_dir / 'submission.csv'

target_column = 'class'
SEED = 2020

In [6]:
df_train = pd.read_csv(train_file, index_col=0)
df_test = pd.read_csv(test_file, index_col=0)

In [7]:
for i in ['u','g','r','i','z']:
    df_train[f'ugriz_{i}'] = df_train[i]
    df_test[f'ugriz_{i}'] = df_test[i]
df_train.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)
df_test.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)

# 변수 생성 

In [8]:
ugriz_col = [c for c in df_train.columns if c.find('ugriz') != -1]
dered_col = [c for c in df_train.columns if c.find('dered') != -1]
airmass_col = [c for c in df_train.columns if c.find('airmass') != -1]

In [9]:
# zip 함수를 이용하여 각 Row별, Magnitude별, max, min, max-min, std, sum을 구한다.
for prefix , g in zip(['ugriz','dered','airmass'], [ugriz_col, dered_col, airmass_col]):
    df_train[f'{prefix}_max'] = df_train[g].max(axis=1)
    df_test[f'{prefix}_max'] = df_test[g].max(axis=1)
    
    df_train[f'{prefix}_min'] = df_train[g].min(axis=1)
    df_test[f'{prefix}_min'] = df_test[g].min(axis=1)
    
    df_train[f'{prefix}_diff'] = df_train[f'{prefix}_max'] - df_train[f'{prefix}_min']
    df_test[f'{prefix}_diff'] = df_test[f'{prefix}_max'] - df_test[f'{prefix}_min']
    
    df_train[f'{prefix}_std'] = df_train[g].std(axis=1)
    df_test[f'{prefix}_std'] = df_test[g].std(axis=1)
    
    df_train[f'{prefix}_sum'] = df_train[g].sum(axis=1)
    df_test[f'{prefix}_sum'] = df_test[g].sum(axis=1)

In [10]:
# diff feature 추가 예: z - i
# itertools combinations을 활용하여 전체 magnitude에서 diff를 구함
# 총 105가지 조합이 나옴. 여기서 안 좋은 것은 permutation importance를 활용하여 제거할 예정
diff_feature = []
for c1, c2 in itertools.combinations(ugriz_col[::-1]+dered_col[::-1]
                                     +airmass_col[::-1],2):
    new_c = f'{c1}_{c2}_diff'
    df_train[new_c] = df_train[c1]-df_train[c2]
    df_test[new_c] = df_test[c1]-df_test[c2]
    diff_feature.append(new_c)
    

In [11]:
# 각 maginitude 별 max-max, min-min, sum-sum 을 구함

for c in itertools.combinations(['ugriz','dered','airmass'],2):
    df_train[f'{c[0]}_{c[1]}_max_diff'] = df_train[f'{c[0]}_max'] - df_train[f'{c[1]}_max']
    df_test[f'{c[0]}_{c[1]}_max_diff'] = df_test[f'{c[0]}_max'] - df_test[f'{c[1]}_max']
    
    df_train[f'{c[0]}_{c[1]}_min_diff'] = df_train[f'{c[0]}_min'] - df_train[f'{c[1]}_min']
    df_test[f'{c[0]}_{c[1]}_min_diff'] = df_test[f'{c[0]}_min'] - df_test[f'{c[1]}_min']
    
    df_train[f'{c[0]}_{c[1]}_sum_diff'] = df_train[f'{c[0]}_sum'] - df_train[f'{c[1]}_sum']
    df_test[f'{c[0]}_{c[1]}_sum_diff'] = df_test[f'{c[0]}_sum'] - df_test[f'{c[1]}_sum']

In [12]:
# 아래와 같은 공식들을 구현함
# http://classic.sdss.org/dr4/algorithms/sdssUBVRITransform.html 

def make_2flux_feature(train, test, c1, c2, func, mag_list=None):
    
    for c in mag_list:
        x=train[f'{c}_{c1}'].values
        y=train[f'{c}_{c2}'].values
        train[f'{c}_{func.__name__}'] = func(x,y)
        
        x=test[f'{c}_{c1}'].values
        y=test[f'{c}_{c2}'].values
        test[f'{c}_{func.__name__}'] = func(x,y)
        
def UB_jester(x1,x2):
    return 0.75*(x1-x2)-0.81

make_2flux_feature(df_train,df_test, 'u','g',UB_jester,['ugriz'])

def BV_jester(x1,x2):
    return 0.62*(x1-x2)+0.15

make_2flux_feature(df_train,df_test, 'g','r',BV_jester,['ugriz'])

def VR_jester(x1,x2):
    return 0.38*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i',VR_jester,['ugriz'])

def RcIc_jester(x1,x2):
    return 0.72*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i', RcIc_jester, ['ugriz'])

def B_jester(x1,x2):
    return x2+0.17*(x1-x2)+0.11

make_2flux_feature(df_train,df_test, 'u','g', B_jester, ['ugriz'])

def V_jester(x1,x2):
    return x1-0.52*(x1-x2)-0.03

make_2flux_feature(df_train,df_test, 'g','r',V_jester, ['ugriz'])

In [13]:
# https://www.sdss.org/dr16/algorithms/segue_target_selection/#Legacy

# 아래 나열된 모든 ugriz 등급은 달리 명시되지 않는 한 dereddened PSF mag입니다.
# dered_X 사용하면 될듯, 일단은 잘 몰라서 ugriz_X 버전과 dered_X 버전을 비교해서 사용하면 될듯.

# ugriz_X 버전

all_data = pd.concat([df_train,df_test], axis=0)
all_data.fillna(-1, inplace=True)

all_data['ugriz_icolor'] = all_data['ugriz_u']*(-0.436) + all_data['ugriz_g']*(1.129) + all_data['ugriz_r']*(-0.119) + all_data['ugriz_i']*(-0.574) + 0.1984

all_data['ugriz_ucolor'] = all_data['ugriz_u']*(-0.249) + all_data['ugriz_g']*(0.794) + all_data['ugriz_r']*(-0.555) + 0.234

all_data['ugriz_p1'] = (all_data['ugriz_u']-all_data['ugriz_g'])*(0.91) + (all_data['ugriz_g']-all_data['ugriz_r'])*(0.415)- 1.280

all_data['ugriz_r_std_div'] = all_data['ugriz_r']/all_data['ugriz_r'].std()

In [14]:
# https://www.sdss.org/dr16/algorithms/legacy_target_selection/
# 멸망과 관련 있는 거라고 함.
# dered_X 가 어떤 곳에서는 멸망과 관련 있는 데이터였음.
# ugriz_X를 붉어짐에 대한 걸로 사용하다면, 여기서는 dered_X를 멸망과 관련 있는 것과 사용하면 될듯

all_data['dered_orthogonal'] = (all_data['dered_r']-all_data['dered_i'])-(all_data['dered_g']-all_data['dered_r'])/4-0.18

all_data['dered_parallel'] = 0.7*(all_data['dered_g']-all_data['dered_r']) + 1.2*((all_data['dered_r']-all_data['dered_i'])-0.18)

In [15]:
# https://www.sdss.org/dr12/algorithms/magnitudes/
# 문서에는 psfMag에 대한 데이터인데, 그것을 나는 ugriz로 사용하고 있으니까 urgiz로 사용

color_list = ['u', 'g', 'r', 'i', 'z']
b_list = [1.4*10e-10, 0.9*10e-10, 1.2*10e-10, 1.8*10e-10, 7.4*10e-10]
f0_list = [24.63, 25.11, 24.80, 24.36, 22.83]
for c, b, f0 in zip(color_list, b_list, f0_list):
    all_data[f'ugriz_{c}_asinh'] = -2.5*np.log(10)*(np.arcsinh((all_data[f'ugriz_{c}']/f0)/(2*b))+np.log(b))

# 데이터셋 분리

In [16]:
# train set
X = all_data.loc[all_data['class'] != -1 , :]
X.drop(columns='class',inplace=True,axis=1)
y = all_data.loc[all_data['class'] != -1, 'class']
y.astype(int)

# test set
test = all_data.loc[all_data['class'] == -1, :]
test.drop(columns='class', inplace=True,axis=1)

# train set split
SEED = 2020
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = SEED)

In [17]:
df_train.shape, df_test.shape, X.shape, y.shape, test.shape

((320000, 154), (80000, 153), (320000, 164), (320000,), (80000, 164))

In [18]:
X.describe()

,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,ugriz_u,ugriz_g,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_min,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_g_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_r_diff,ugriz_i_ugriz_g_diff,ugriz_i_ugriz_u_diff,...,dered_r_airmass_r_diff,dered_r_airmass_g_diff,dered_r_airmass_u_diff,dered_g_dered_u_diff,dered_g_airmass_z_diff,dered_g_airmass_i_diff,dered_g_airmass_r_diff,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_r_diff,dered_u_airmass_g_diff,dered_u_airmass_u_diff,airmass_z_airmass_i_diff,airmass_z_airmass_r_diff,airmass_z_airmass_g_diff,airmass_z_airmass_u_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_min_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,ugriz_UB_jester,ugriz_BV_jester,ugriz_VR_jester,ugriz_RcIc_jester,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,ugriz_r_std_div,dered_orthogonal,dered_parallel,ugriz_u_asinh,ugriz_g_asinh,ugriz_r_asinh,ugriz_i_asinh,ugriz_z_asinh
count,3.2000e+05,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,3.2000e+05,3.2000e+05,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,...,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000
mean,5.8308e-02,19.6289,18.2271,17.6154,17.1523,16.8693,6.3327,6.1388,1.1758,1.1765,1.1751,1.1754,1.1761,19.8524,18.3768,17.7358,17.1742,16.8752,19.8858,16.8283,3.0575,1.2601,90.0144,19.6647,16.8154,2.8493,1.1668,89.4930,1.1792,1.1725,6.7141e-03,2.6542e-03,5.8790,-0.2990,-0.8606,-1.5017,-2.9772,0.0059,-0.2772,-0.7402,-1.3519,-2.7537,15.6991,15.6998,15.7001,15.6987,15.6994,-0.5615,-1.2026,-2.6782,...,16.4403,16.4389,16.4397,-1.4018,17.0510,17.0517,17.0520,17.0506,17.0513,18.4527,18.4534,18.4537,18.4523,18.4531,0.0007,0.0010,-0.0004,0.0004,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.2211,0.0129,0.5214,18.7066,15.6559,84.1355,18.4855,15.6429,83.6140,0.2967,0.5475,0.4834,0.6743,18.7377,18.0135,0.3216,0.0386,0.3288,12.0757,0.1302,0.7679,1.2692,1.8050,1.9488,1.9881,1.7000
std,5.7546e-01,1.9236,17.7866,1.4730,25.0758,35.4365,8.8811,8.5516,0.1163,0.1181,0.1147,0.1155,0.1172,1.9455,31.1108,1.4662,52.5615,57.1463,1.9736,57.1440,57.1509,30.4830,128.1521,1.9280,35.4343,35.4410,17.7147,68.9525,0.1184,0.1143,5.8405e-03,2.3083e-03,0.5816,22.4346,57.1370,47.9608,57.1518,28.2757

In [19]:
test.describe()

,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,ugriz_u,ugriz_g,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_min,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_g_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_r_diff,ugriz_i_ugriz_g_diff,ugriz_i_ugriz_u_diff,...,dered_r_airmass_r_diff,dered_r_airmass_g_diff,dered_r_airmass_u_diff,dered_g_dered_u_diff,dered_g_airmass_z_diff,dered_g_airmass_i_diff,dered_g_airmass_r_diff,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_r_diff,dered_u_airmass_g_diff,dered_u_airmass_u_diff,airmass_z_airmass_i_diff,airmass_z_airmass_r_diff,airmass_z_airmass_g_diff,airmass_z_airmass_u_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_min_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,ugriz_UB_jester,ugriz_BV_jester,ugriz_VR_jester,ugriz_RcIc_jester,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,ugriz_r_std_div,dered_orthogonal,dered_parallel,ugriz_u_asinh,ugriz_g_asinh,ugriz_r_asinh,ugriz_i_asinh,ugriz_z_asinh
count,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,8.0000e+04,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,...,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000
mean,6.0083e-02,19.6350,18.2638,17.6193,17.2178,16.8702,6.3499,6.1578,1.1760,1.1767,1.1753,1.1756,1.1763,19.8598,18.4371,17.7397,17.3065,17.1490,19.9839,17.0133,2.9706,1.2082,90.4921,19.6729,16.8157,2.8572,1.1640,89.6061,1.1794,1.1727,6.7318e-03,2.6612e-03,5.8799,-0.1575,-0.5908,-1.2882,-2.7109,0.2787,-0.0689,-0.4704,-1.1148,-2.4860,15.9726,15.9733,15.9736,15.9722,15.9730,-0.4333,-1.1306,-2.5533,...,16.4440,16.4426,16.4434,-1.3712,17.0875,17.0882,17.0885,17.0870,17.0878,18.4586,18.4593,18.4596,18.4582,18.4590,0.0007,0.0010,-0.0004,0.0004,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.3109,0.1975,0.8860,18.8045,15.8406,84.6122,18.4936,15.6431,83.7262,0.2571,0.5824,0.4346,0.5819,18.7890,18.0445,0.3100,0.0824,0.3041,12.0784,0.0604,0.7170,1.2678,1.8060,1.9531,1.9843,1.6981
std,3.4684e-01,1.9210,1.6657,1.4760,1.3324,35.4367,8.8728,8.5509,0.1164,0.1182,0.1148,0.1156,0.1173,1.9543,1.7030,1.4786,1.3207,24.6431,24.6764,1.2616,24.6651,11.0261,25.5608,1.9296,35.4342,35.4250,15.8383,36.2002,0.1185,0.1144,5.8600e-03,2.3160e-03,0.5822,24.6299,24.6382,24.6502,24.6773,60.0184,24.6307,24.6389,24.6522,24.6755,24.6420,24.6420,24.6420,24.6419,24.6420,0.5820,0.9922,1.4322,...,1.47

# feature 선택

In [20]:
# Xgboost
xgb_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'tree_method': 'auto',
    'objective': 'multi:softmax',
    'num_class': 3,
    'random_state': 2020
}

In [21]:
xgb_clf = xgb.XGBClassifier(**xgb_params)
xgb_clf.fit(X_train,y_train)
p = xgb_clf.predict(X_val)
print(accuracy_score(p,y_val))

NameError: name 'X_val' is not defined

In [24]:
p = xgb_clf.predict(X_test)
print(accuracy_score(p,y_test))

0.9332604166666667


In [25]:
perm = PermutationImportance(xgb_clf, random_state=2020).fit(X_test,y_test)

In [29]:
eli5.show_weights(perm, top = 200, feature_names = X_test.columns.tolist())

Weight,Feature
0.4388 ± 0.0029,redshift
0.0590 ± 0.0024,dered_orthogonal
0.0179 ± 0.0010,dered_g_dered_u_diff
0.0034 ± 0.0006,ugriz_r_dered_u_diff
0.0029 ± 0.0005,dered_r_dered_u_diff
0.0028 ± 0.0005,dered_z_dered_i_diff
0.0027 ± 0.0003,ugriz_ucolor
0.0023 ± 0.0006,ugriz_icolor
0.0018 ± 0.0004,dered_r_dered_g_diff
0.0010 ± 0.0004,ugriz_r_ugriz_g_diff


In [43]:
feature_importances=pd.DataFrame(perm.feature_importances_, index=X_test.columns.tolist())

In [1]:
top = feature_importances.loc[feature_importances[0]>0.0003]

NameError: name 'feature_importances' is not defined

In [2]:
top.shape

NameError: name 'top' is not defined

In [59]:
best_feature = top.index.tolist()

In [61]:
print(best_feature)

['redshift', 'dered_g', 'dered_r', 'dered_i', 'airmass_g', 'ugriz_r', 'ugriz_i', 'airmass_diff', 'ugriz_z_ugriz_g_diff', 'ugriz_z_ugriz_u_diff', 'ugriz_z_dered_z_diff', 'ugriz_z_dered_g_diff', 'ugriz_i_ugriz_r_diff', 'ugriz_i_dered_g_diff', 'ugriz_r_ugriz_g_diff', 'ugriz_r_ugriz_u_diff', 'ugriz_r_dered_z_diff', 'ugriz_r_dered_r_diff', 'ugriz_r_dered_g_diff', 'ugriz_r_dered_u_diff', 'ugriz_g_ugriz_u_diff', 'ugriz_g_dered_g_diff', 'ugriz_g_dered_u_diff', 'ugriz_u_dered_r_diff', 'ugriz_u_dered_g_diff', 'dered_z_dered_i_diff', 'dered_z_airmass_g_diff', 'dered_r_dered_g_diff', 'dered_r_dered_u_diff', 'dered_g_dered_u_diff', 'dered_g_airmass_i_diff', 'dered_g_airmass_g_diff', 'dered_u_airmass_i_diff', 'airmass_z_airmass_g_diff', 'ugriz_dered_min_diff', 'dered_airmass_sum_diff', 'ugriz_icolor', 'ugriz_ucolor', 'ugriz_p1', 'dered_orthogonal', 'dered_parallel']
